In [1]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT
import torch
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)

In [26]:
piece_dict = {'Tu': 0, 'Tr': 1, 'Td': 2, 'Tl': 3, 'Jl': 4, 'Ju': 5, 'Jr': 6, 'Jd': 7, 'Zh': 8, 'Zv': 9, 'O': 10, 'Sh': 11, 'Sv': 12, 'Lr': 13, 'Ld': 14, 'Ll': 15, 'Lu': 16, 'Iv': 17, 'Ih': 18}

def process_board(board):
    # Returns board, holes and boundaries
    board = np.where(board == 239,0,1)
    holes = np.zeros(board.shape[1], dtype=int)
    boundaries = np.zeros(board.shape[1], dtype=int)

    for col in range(board.shape[1]):
        column_data = board[:, col]
        first_block_idx = np.where(column_data == 1)[0]
        if first_block_idx.size > 0:
            first_block_idx = first_block_idx[0]
            boundaries[col] = board.shape[0] - first_block_idx
            holes[col] = np.count_nonzero(column_data[first_block_idx:] == 0)
    
    return board,holes,boundaries

def concat_data(holes,boundaries,cur_piece,cleared_lines):
    combined_array = np.concatenate((holes, boundaries))
    final_array = np.append(combined_array, [cur_piece, cleared_lines])
    return final_array

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    cur_piece = piece_dict[info["current_piece"]]
    cleared_lines = info["number_of_lines"]
    board, holes, boundaries = process_board(env.board)
    state_data = concat_data(holes,boundaries,cur_piece,cleared_lines)
    state_data = state_data.copy()
    state_data = torch.tensor(state_data, dtype=torch.float32)
    